In [1]:
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
from langchain.document_loaders import TextLoader

/home/bssayla/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [6]:
model_name='google/gemma-7b-it'

model_config = transformers.AutoConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True

bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_quant_type="nf4",bnb_4bit_compute_dtype=torch.float16,bnb_4bit_use_double_quant=True)

KeyError: 'gemma'

In [7]:
from pdfminer.high_level import extract_text as extract_text_pdf_miner
from langchain.text_splitter import RecursiveCharacterTextSplitter
persist_directory = "/content/knowledgebase"
from langchain.chains import VectorDBQA, RetrievalQA,ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain import PromptTemplate, LLMChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.docstore.document import Document
embedding = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2048,
    output_scores=True
)

GemmaLLM = HuggingFacePipeline(pipeline=text_generation_pipeline)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2024-05-18 01:56:11.383959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 01:56:11.384058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 01:56:11.543185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain

In [8]:
with open ("/kaggle/input/lawmorocco/output.txt", "r") as myfile:
    data = myfile.read().splitlines()

docs = []
splitter = RecursiveCharacterTextSplitter(chunk_size=1024)
for chunk in splitter.split_text(data[0]):
    docs.append(Document(page_content=chunk, metadata={"source": "Moroccan law"}))

vector_collection_fixed_size = Chroma.from_documents(documents=docs, persist_directory=persist_directory, embedding=embedding)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/lawmorocco/output.txt'

In [14]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [15]:
vectordb.similarity_search("عقوبة الرشوة فالمغرب",k=5)

[Document(page_content='the Islamic region the monarchical regime or incites to attack on the territorial integrity of the Kingdom The penalty incurred is increased to two years to five years of imprisonment and a fine of 50000 to 500000 dirhams or to one of its penalties only when the acts referred to in the first paragraph above are committed either by speech cries or threats uttered in public places or meetings or by posters exposed to public view either by sale distribution or any means meeting the condition of publicity including electronically on paper and by audiovisual means     90 Article 1 of Dahir 100284 of 19 Rejeb 1421 October 17 2000 relating to coat of arms of the Kingdom Official Bulletin No 4844 of 5 Chaabane 1421 November 12 2000 p 927 provides that The coat of arms of Our Kingdom is defined as follows Gules in chief a half moving sun has 15 rays of gold on a ground of azure supported by a divided fess vaulted in Vert tapered in Gold and Argent all overcharged with a 

In [18]:
memory = ConversationBufferMemory(return_messages=True,memory_key='chat_history')
retriever = vectordb.as_retriever(search_kwargs={"k": 10})
qa_chain = ConversationalRetrievalChain.from_llm(llm=GemmaLLM,retriever = retriever,memory=memory)

In [19]:
qa_chain = ConversationalRetrievalChain.from_llm(llm=GemmaLLM,retriever = retriever,memory=memory)

In [20]:
query = "عقوبة الرشوة فالمغرب"
response = qa_chain({'question':query,'chat_history':[]})
response['answer']

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nthe Islamic region the monarchical regime or incites to attack on the territorial integrity of the Kingdom The penalty incurred is increased to two years to five years of imprisonment and a fine of 50000 to 500000 dirhams or to one of its penalties only when the acts referred to in the first paragraph above are committed either by speech cries or threats uttered in public places or meetings or by posters exposed to public view either by sale distribution or any means meeting the condition of publicity including electronically on paper and by audiovisual means     90 Article 1 of Dahir 100284 of 19 Rejeb 1421 October 17 2000 relating to coat of arms of the Kingdom Official Bulletin No 4844 of 5 Chaabane 1421 November 12 2000 p 927 provides that The coat of arms of Our Kingdom is defined as follows Gules in chief a half mov